# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset
data = pd.read_csv("pokemon.csv")
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
# your code here
data['Total_calculated']=data['HP']+data['Attack']+data['Defense']+data['Sp. Atk']+data['Sp. Def']+data['Speed']
(data['Total_calculated']-data['Total']).max()  
# the 'Total_calculated' column matches exactly the 'Total' column, so 'Total' is the sum of the considered columns
# and the formula works for all pokemon.

0

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [4]:
# your code here
data=data.drop(['#', 'Name', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary', 'Total_calculated'], axis=1)
data['Type 2'] = data['Type 2'].fillna('None')
data.head()

,Type 1,Type 2,Total
0,Grass,Poison,318
1,Grass,Poison,405
2,Grass,Poison,525
3,Grass,Poison,625
4,Fire,None,309


In [5]:
y = data['Total']
X = data.drop(['Total'], axis=1)

In [6]:
X_num = X.select_dtypes(include = np.number)
X_cat = X.select_dtypes(include = object)

In [7]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(X_cat)

In [8]:
cols=[]
for row in encoder.categories_:
    for col_name in row[1::]:
        cols.append(col_name)
cols=encoder.get_feature_names(input_features=X_cat.columns)
X_cat_encode=pd.DataFrame(encoder.transform(X_cat).toarray(),columns=cols)
X_cat_encode.head()

,Type 1_Dark,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,Type 1_Flying,Type 1_Ghost,Type 1_Grass,Type 1_Ground,...,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_None,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [14]:
# your code here
data=pd.concat([X_num, X_cat_encode, y], axis=1) 
correlations_matrix = data.corr()
correlations_matrix

,Type 1_Dark,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,Type 1_Flying,Type 1_Ghost,Type 1_Grass,Type 1_Ground,...,Type 2_Ground,Type 2_Ice,Type 2_None,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Total
Type 1_Dark,1.000000,-0.040984,-0.048438,-0.029584,-0.037524,-0.052938,-0.014233,-0.040984,-0.062173,-0.040984,...,-0.042946,0.071991,-0.064256,-0.014233,-0.042300,0.023490,-0.026796,0.045446,-0.026796,0.017818
Type 1_Dragon,-0.040984,1.000000,-0.049245,-0.030077,-0.038149,-0.053820,-0.014470,-0.041667,-0.063209,-0.041667,...,0.112272,0.118700,-0.056679,-0.014470,-0.043005,0.085964,-0.027243,-0.034325,-0.027243,0.196532
Type 1_Electric,-0.048438,-0.049245,1.000000,-0.035547,-0.045088,-0.063609,-0.017102,-0.049245,-0.074706,-0.049245,...,-0.051602,0.009617,0.063312,0.138368,-0.050827,-0.050041,-0.032197,0.060014,0.009617,0.016715
Type 1_Fairy,-0.029584,-0.030077,-0.035547,1.000000,-0.027538,-0.038850,-0.010445,-0.030077,-0.045628,-0.030077,...,-0.031517,-0.019665,0.117907,-0.010445,-0.031043,-0.030563,-0.019665,-0.024778,-0.019665,-0.026948
Type 1_Fighting,-0.037524,-0.038149,-0.045088,-0.027538,1.000000,-0.049277,-0.013248,-0.038149,-0.057873,-0.038149,...,-0.039976,-0.024943,0.096586,-0.013248,-0.039375,0.065654,-0.024943,0.053226,-0.024943,-0.029086
Type 1_Fire,-0.052938,-0.053820,-0.063609,-0.038850,-0.049277,1.000000,-0.018691,-0.053820,-0.081647,-0.053820,...,0.017973,-0.035189,0.029528,0.125084,-0.055549,-0.003697,0.003480,-0.013332,0.003480,0.050527
Type 1_Flying,-0.014233,-0.014470,-0.017102,-0.010445,-0.013248,-0.018691,1.000000,-0.014470,-0.021951,-0.014470,...,-0.015163,-0.009461,0.002483,-0.005025,-0.014935,-0.014704,-0.009461,-0.011921,-0.009461,0.029504
Type 1_Ghost,-0.040984,-0.041667,-0.049245,-0.030077,-0.038149,-0.053820,-0.014470,1.000000,-0.063209,-0.041667,...,-0.043661,-0.027243,-0.069445,-0.014470,0.083480,-0.042340,-0.027243,-0.034325,-0.027243,0.007594
Type 1_Grass,-0.062173,-0.063209,-0.074706,-0.045628,-0.057873,-0.081647,-0.021951,-0.063209,1.000000,-0.063209,...,-0.044608,0.059883,-0.006861,-0.021951,0.263700,-0.019742,-0.041328,0.002029,-0.041328,-0.036057
Type 1_Ground,-0.040984,-0.041667,-0.049245,-0.030077,-0.038149,-0.053820,-0.014470,-0.041667,-0.063209,1.000000,...,-0.043661,-0.027243,-0.031148,-0.014470,-0.043005,0.021812,0.118700,0.004681,-0.027243,0.004082


In [15]:
display(correlations_matrix.nlargest(5, ['Total']))

# 'Type 1'= Dragon has the highest correlation with 'Total'.

,Type 1_Dark,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,Type 1_Flying,Type 1_Ghost,Type 1_Grass,Type 1_Ground,...,Type 2_Ground,Type 2_Ice,Type 2_None,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Total
Total,0.017818,0.196532,0.016715,-0.026948,-0.029086,0.050527,0.029504,0.007594,-0.036057,0.004082,...,0.016486,0.100870,-0.185945,-0.013956,-0.067837,0.076054,-0.000512,0.070307,-0.018800,1.000000
Type 1_Dragon,-0.040984,1.000000,-0.049245,-0.030077,-0.038149,-0.053820,-0.014470,-0.041667,-0.063209,-0.041667,...,0.112272,0.118700,-0.056679,-0.014470,-0.043005,0.085964,-0.027243,-0.034325,-0.027243,0.196532
Type 2_Fighting,0.036251,-0.037412,-0.044216,-0.027006,-0.034254,0.151836,-0.012992,-0.037412,0.018087,-0.037412,...,-0.039203,-0.024461,-0.176974,-0.012992,-0.038614,-0.038017,-0.024461,-0.030820,-0.024461,0.138726
Type 2_Dragon,0.100556,-0.030969,0.000370,-0.022355,-0.028355,-0.005812,0.228243,0.055056,-0.017152,0.055056,...,-0.032452,-0.020248,-0.146496,-0.010755,-0.031964,-0.031470,-0.020248,-0.025513,-0.020248,0.115240
Type 2_Ice,0.071991,0.118700,0.009617,-0.019665,-0.024943,-0.035189,-0.009461,-0.027243,0.059883,-0.027243,...,-0.028547,1.000000,-0.128868,-0.009461,-0.028118,-0.027683,-0.017812,-0.022443,-0.017812,0.100870


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [16]:
# your code here
# 'Type 1'= Dragon and 'Type 2' = Fighting have the highest correlations with 'Total'. So these would be the selected types.
# This combination is not available in the DataFrame
data = pd.read_csv("pokemon.csv")

In [17]:
data[((data['Type 1']=='Dragon') & (data['Type 2']=='Fighting'))]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
